---

## 네이버 주식정보 가져오기
https://finance.naver.com/sise/

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
url = 'https://finance.naver.com/sise/'
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')
type(soup)

#### id를 이용해서 정보를 가져올 수 있다.
#### tag : span
#### id : KOSPI_now

In [ ]:
kospi = soup.find('span', id='KOSPI_now').text
kosdaq = soup.find('span', id='KOSDAQ_now').text
kospi_200 = soup.find('span', id='KPI200_now').text

print('현재 코스피 지수 :', kospi)
print('현재 코스닥 지수 :', kosdaq)
print('현재 코스피200 지수 :', kospi_200)

In [ ]:
### 코스피 200지수
soup_p = soup.find('div', id='tab_sel3_risefall').find('dd', class_='profit').text
list_soup = [i for i in soup_p.split('\n') if i]

print(list_soup[0]+'은', list_soup[1]+'입니다.')

soup_p = soup.find('div', id='tab_sel3_risefall').find('dd', class_='loss').text
list_soup = [i for i in soup_p.split('\n') if i]

print(list_soup[0]+'은', list_soup[1]+'입니다.')

soup_p = soup.find('div', id='tab_sel3_risefall').find('dd', class_='total').text
list_soup = [i for i in soup_p.split('\n') if i]

print(list_soup[0]+'는', list_soup[1]+'입니다.')

In [ ]:
### 코스닥 지수
soup_p = soup.find('div', id='tab_sel2_risefall').find('dd', class_='profit').text
list_soup = soup_p.split('\n')

print(list_soup[1]+'은', list_soup[2]+'입니다.')

soup_p = soup.find('div', id='tab_sel2_risefall').find('dd', class_='loss').text
list_soup = soup_p.split('\n')

print(list_soup[1]+'은', list_soup[2]+'입니다.')

soup_p = soup.find('div', id='tab_sel2_risefall').find('dd', class_='total').text
list_soup = soup_p.split('\n')

print(list_soup[1]+'는', list_soup[2]+'입니다.')

In [ ]:
### 코스피 지수
soup_t = soup.find('dl', id='basis_trend').text
list_soup_t = [i for i in soup_t.split('\n') if i]

print(list_soup_t[1]+'은', list_soup_t[2]+'입니다.')
print(list_soup_t[3]+'은', list_soup_t[4]+'입니다.')
print(list_soup_t[5]+'는', list_soup_t[6]+'입니다.')

---
정리

---

In [ ]:
url = "https://finance.naver.com/sise/"

page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

rank_1 = soup.find('div', class_='rgt')
rank_1 = soup.findAll('ul', id='popularItemList')

list_rank = rank_1[0].findAll('li')

for i in list_rank:
    code = i.find('a')
    company = code.text
    code = code.get('href')
    print(company+'의 코드 : ', code[-6:])

---
Top 종목에 있는 정보를 sub메뉴(상한가, 하한가 등) 정보를 가져와서 csv파일로 정리하기.

---

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

url = "https://finance.naver.com/sise/"
sub_url = "https://finance.naver.com/item/main.nhn?code="   # 인기 종목의 각 url 공통부분
comp_list = []
code_list = []
target_data3 = []
total_info = []

page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

rank_1 = soup.find('div', class_='rgt')
rank_1 = soup.findAll('ul', id='popularItemList')

list_rank = rank_1[0].findAll('li')

for i in list_rank:   ## 인기 검색 종목의 코드 추출
    code = i.find('a').get('href')
    company = i.find('a').text
    comp_list.append(company)
    code_list.append(code[-6:])    
#     print(company, code[-6:])

for k in range(len(comp_list)):  ## 인기 검색 종목 각각의 url에서 정보 추출
    target_url = sub_url + str(code_list[k])
    target_page = urlopen(target_url)
    target_soup = BeautifulSoup(target_page, 'html.parser')
    
    ## 각 url의 정보 추출
    target_data1 = target_soup.find('div', id="middle", class_="new_totalinfo").findAll('dd')
    
    ## target_data3 : 각 url의 11가지 정보를 리스트 형태로 저장한 것
    for j in range(1, 12):   # 0~12 인덱스 : 정보
        target_data2 = target_data1[j].text
        target_data3.append(target_data2)


In [ ]:
## target_data3의 각 인덱스의 정보를 목록과 값으로 분리
c_1, c_2, c_3, c_4, c_5, c_6  = [], [], [], [], [], []
c_7, c_8, c_9, c_10, c_11,  = [], [], [], [], []

for j in target_data3:
    target_data4 = j.split(' ')
    target_data5 = ' '.join(target_data4[1:])
    total_info.append(target_data5)   # total_info : 10개 종목 당 11개 정보 / 리스트 형태

for j in range(0, 110, 11):   # 종목명
    c_1.append(total_info[j])
for j in range(1, 110, 11):   # 종목코드
    c_2.append(total_info[j])
for j in range(2, 110, 11):   # 현재가
    c_3.append(total_info[j])
for j in range(3, 110, 11):   # 전일가
    c_4.append(total_info[j])
for j in range(4, 110, 11):   # 시가
    c_5.append(total_info[j])
for j in range(5, 110, 11):   # 고가
    c_6.append(total_info[j])
for j in range(6, 110, 11):   # 상한가
    c_7.append(total_info[j])
for j in range(7, 110, 11):   # 저가
    c_8.append(total_info[j])
for j in range(8, 110, 11):   # 하한가
    c_9.append(total_info[j])
for j in range(9, 110, 11):   # 거래량
    c_10.append(total_info[j])
for j in range(10, 110, 11):   # 거래대금
    c_11.append(total_info[j])
    
# print(c_1)
# print(c_2)
# print(c_3)
# print(c_4)
# print(c_5)
# print(c_6)
# print(c_7)
# print(c_8)
# print(c_9)
# print(c_10)
# print(c_11)

In [ ]:
dat = {'종목명':c_1, '종목코드':c_2, '현재가':c_3 , '전일가':c_4 , '시가':c_5 ,
       '고가':c_6 , '상한가':c_7 , '저가':c_8 , '하한가':c_9 , '거래량':c_10 ,
       '거래대금':c_11 }
dat = pd.DataFrame(dat)
dat

In [ ]:
dat.to_csv("인기종목 정보.csv", index=False)

## 확인
import os
print(os.getcwd())  # 현재 위치
print(os.listdir(os.getcwd()))

#### 관심 있는 종목 5개 선정, 코드 확인 후, 내가 필요로 하는 정보 6개 가져오기

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

comp_list = ['삼성전자', '카카오', '현대바이오', '셀트리온', 'NAVER']
code_list = ['005930', '035720', '048410', '068270', '035420']
list_1 = []

base_url = 'https://finance.naver.com/item/main.nhn?code='

for idx, code in enumerate(code_list):
    all_url = base_url + code
    page = urlopen(all_url)
    soup = BeautifulSoup(page, 'lxml')
    
    cur_price = soup.find('p', class_='no_today').span.text
    am = soup.find('div', id="middle", class_="new_totalinfo").findAll('dd')
    
    print(comp_list[idx], cur_price, '\n', am[10].text, '\n', 
         am[3].text, '\n', am[4].text, '\n', am[5].text, '\n', am[6].text, '\n',
         am[7].text, '\n', am[8].text, '\n', am[9].text)
    
    print('\n')